In [ ]:
##1
import cv2
import tensorflow.keras as keras
import tensorflow as tf
import numpy as np
import os
import cv2
from google.colab.patches import cv2_imshow
import random

shape = (125, 50)
#50, 125

In [ ]:
persons = []
real_dir = '/content/drive/My Drive/SOAI/Ass 2 /dataset2/real'
fake_dir= '/content/drive/My Drive/SOAI/Ass 2 /dataset2/forge'

w =[]
h =[]


###Storing and Resizing the Images in corresponding variables. Different variables have the following meanings:
real001  -->> list of Image of real signatures of person 001


real002  -->> list of Image of real signatures of person 002


.


.


.


real005  -->> list of Image of real signatures of person 005

Fake001  -->> list of Image of fake signatures of person 001


'


'


'


'


fake005   -->> list of Image of fake signatures of person 005


Data has been downloaded from the following Kaggle Dataset into my Gdrive - https://www.kaggle.com/divyanshrai/handwritten-signatures

I chose this data because I could not find any free publicly available Dataset forthe same. 





## 001

In [ ]:
ls = os.listdir(real_dir)
real001 = []
for i in range(len(ls)):
  # list 
  if ls[i][0:3] in persons:
    pass
  else:
    persons.append(ls[i][0:3])
  #list
  if(ls[i][0:3] == '001'):
    real001.append(cv2.resize(cv2.imread(real_dir+'/'+ls[i]), shape, interpolation = cv2.INTER_AREA))

len(persons)

14

In [ ]:
#+32, 61, 88
ls = os.listdir(fake_dir)
fake001 = []
for i in range(len(ls)):
  # list 
  if ls[i][0:3] in persons:
    pass
  else:
    persons.append(ls[i][0:3])
  #list
  if(ls[i].find('001', 3) != -1):
    fake001.append(cv2.resize(cv2.imread(fake_dir + '/'+ls[i]), shape, interpolation = cv2.INTER_AREA))
persons.sort()
persons


['001',
 '002',
 '003',
 '004',
 '005',
 '006',
 '007',
 '008',
 '009',
 '010',
 '011',
 '012',
 '021',
 '022']

##002

In [ ]:
ls = os.listdir(real_dir)
real002 = []
for i in range(len(ls)):
  if(ls[i][0:3] == '002'):
    real002.append(cv2.resize(cv2.imread(real_dir+'/'+ls[i]), shape, interpolation = cv2.INTER_AREA))


ls = os.listdir(fake_dir)
fake002 = []
for i in range(len(ls)):
  if(ls[i].find('002', 3) != -1):
    fake002.append(cv2.resize(cv2.imread(fake_dir +'/'+ls[i]), shape, interpolation = cv2.INTER_AREA))



## 003

In [ ]:
ls = os.listdir(real_dir)
real003 = []
for i in range(len(ls)):
  if(ls[i][0:3] == '003'):
    real003.append(cv2.resize(cv2.imread(real_dir +'/'+ls[i]), shape, interpolation = cv2.INTER_AREA))


ls = os.listdir(fake_dir)
fake003 = []
for i in range(len(ls)):
  if(ls[i].find('003', 3) != -1):
    fake003.append(cv2.resize(cv2.imread(fake_dir +'/'+ls[i]), shape, interpolation = cv2.INTER_AREA))



#004

In [ ]:
ls = os.listdir(real_dir)
real004 = []
for i in range(len(ls)):
  if(ls[i][0:3] == '004'):
    real004.append(cv2.resize(cv2.imread(real_dir+'/'+ls[i]), shape, interpolation = cv2.INTER_AREA))


ls = os.listdir(fake_dir)
fake004 = []
for i in range(len(ls)):
  if(ls[i].find('004', 3) != -1):
    fake004.append(cv2.resize(cv2.imread(fake_dir +'/'+ls[i]), shape, interpolation = cv2.INTER_AREA))



##005

In [ ]:
ls = os.listdir(real_dir)
real005 = []
for i in range(len(ls)):
  if(ls[i][0:3] == '005'):
    real005.append(cv2.resize(cv2.imread(real_dir +'/'+ls[i]), shape, interpolation = cv2.INTER_AREA))


ls = os.listdir(fake_dir)
fake005 = []
for i in range(len(ls)):
  if(ls[i].find('005', 3) != -1):
    fake005.append(cv2.resize(cv2.imread(fake_dir +'/'+ls[i]), shape, interpolation = cv2.INTER_AREA))



#METRIC LEARNING APPROACH


Here we are going to use the Architecture used for Face Recognition model. The Network will take a set of three images (real signature of a person, another real signature of same person, fake signature of that person), and output three encoded vectors of length 500 each, encoding the features of each image. 

The good thing about the mode is that it is not writer specific, i.e., once it is trained it can differentiate between real and fake signatures of any other person who might not be in the training set. 

on the other hand, the downside of the model is that it requires fake signatures of persons for training, which are less practical way of approaching the problem.


### DATA PREPARATION

So, the input data is created as sets of three images in the manner described above.

In [ ]:
set1 = []
set2 = []
set3 = []
Y = []

for real, fake in zip([real001, real002, real003,real004, real005], [fake001, fake002, fake003, fake004, fake005]):
  for i in range(20):
    r = np.random.randint(0, high=len(real), size=2)
    set1.append(real[r[0]])
    set2.append(real[r[1]])
    r = np.random.randint(0, high=len(fake), size=1)
    set3.append(fake[r[0]])

set1 = np.asarray(set1)
set2 = np.asarray(set2)
set3 = np.asarray(set3)
#set1 = set1/255 - 0.5
#set2 = set2/255 - 0.5
Y = np.zeros((set1.shape[0], 500,3))
print(set1.shape, set2.shape)

(100, 50, 125, 3) (100, 50, 125, 3)


###ARCHITECTURE

The model consists of two parts namely, encoder and comparator

ENCODER : takes three images, runns CNN over all of them and finally outputs three 500 dimensional vector for each image, encoding the image features. Loss function aims to reduce the gap between outputs of real signatures pair , and widen the gap between vectors of real and fake image pair.
It is trained on the data generated above.

COMPARATOR : This takes two images, one of real signature, and other of the signature which is to be tested against it. Thiese images arre passed through the encoder above, and their freatures are extracted. These are then passed through another neural network of densely connected layers which outputs a number between 0 and 1 signifying closeness. i.e. the more close to 0, the more is the second signature is matching to the real one. 

This is trained by keeping the encoder weights fixed and training the densely connected neural- network ahead of it. 

._. I tried to train the whole network at once, but it did not work out well. The reason might be that the weights of the ENCODER, which are trained using Approach for Face Recognition, are trained to differentiate between the images. But training them from comparatoe point of view, might force it to leave this behaviour and assume some other behaviour which is not able to differentiate between very similar images(as all fake signatures are). 

### Siamese Network Model for feature Encoding (ENCODER MODEL CREATION AND TRAINING)

We use pretraines VGG 16 model for creating a part of the encoder.


In [ ]:
premodel = keras.applications.VGG16(include_top=False,weights="imagenet", input_shape=(50,125,3))

58892288/58889256 [==============================] - 2s 0us/step


In [ ]:
def added_model():
  inp = keras.layers.Input(1536)
  D = keras.layers.Dense(500)(inp)
  return keras.Model(inp, D)

In [ ]:
added_model = added_model()

In [ ]:
def model():
  inp1 = keras.layers.Input(shape=(50,125,3))
  inp2 = keras.layers.Input(shape=(50,125,3))
  inp3 = keras.layers.Input(shape=(50,125,3))

  out1 = premodel(inp1)
  out1 = keras.layers.Flatten()(out1)
  out1 = added_model(out1)

  out2 = premodel(inp2)
  out2 = keras.layers.Flatten()(out2)
  out2 = added_model(out2)
  
  out3 = premodel(inp3)
  out3 = keras.layers.Flatten()(out3)
  out3 = added_model(out3)

  out = keras.layers.Lambda(lambda x: tf.concat(x, axis = 2))([tf.expand_dims(out1, -1), tf.expand_dims(out2, -1), tf.expand_dims(out3, -1)])
  model = keras.Model([inp1, inp2, inp3], out)
  return model

In [ ]:
encoder = model()

In [ ]:
encoder.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 50, 125, 3)] 0                                            
__________________________________________________________________________________________________
input_4 (InputLayer)            [(None, 50, 125, 3)] 0                                            
__________________________________________________________________________________________________
input_5 (InputLayer)            [(None, 50, 125, 3)] 0                                            
__________________________________________________________________________________________________
vgg16 (Model)                   (None, 1, 3, 512)    14714688    input_3[0][0]                    
                                                                 input_4[0][0]              

In [ ]:
def loss_fn(y_true, y_pred):
  loss_same = tf.reduce_mean(tf.math.square(tf.math.subtract(y_pred[:,:, 0] , y_pred[:,:,1])))
  loss_diff = -tf.reduce_mean(tf.math.square(tf.math.subtract(y_pred[:,:, 0] , y_pred[:,:,2])))
  loss = loss_diff + loss_same
  loss = 1/(1+tf.math.exp(-loss))
  return loss

In [ ]:

#encoder = keras.models.load_model('/content/drive/My Drive/SOAI/Ass 2 /encoder.hdf5', custom_objects={'loss_fn': loss_fn})
premodel = keras.models.load_model('/content/drive/My Drive/SOAI/Ass 2 /premodel.hdf5')
added_model = keras.models.load_model('/content/drive/My Drive/SOAI/Ass 2 /added_model.hdf5')

#encoder = keras.Model(premodel.input, D1(premodel.output))
#m = keras.models.load_model('/content/drive/My Drive/SOAI/Ass 2 /m.hdf5', custom_objects={'loss_fn': loss_fn})

In [ ]:
encoder = model()

In [ ]:
encoder.compile(loss = loss_fn, optimizer = 'Adam')

In [ ]:
for i in range(1000):
  print(i)
  encoder.fit([set1, set2, set3], Y, batch_size=set1.shape[0], epochs = 10)
  encoder.save('/content/drive/My Drive/SOAI/Ass 2 /encoder.hdf5')
  premodel.save('/content/drive/My Drive/SOAI/Ass 2 /premodel.hdf5')
  added_model.save('/content/drive/My Drive/SOAI/Ass 2 /added_model.hdf5')

###Comparor

### Data Preparation for Comparor

In [ ]:
set1 = []
set2 = []

Y = []
# real vs real
for real, fake in zip([real001, real002, real003,real004, real005], [fake001, fake002, fake003, fake004, fake005]):
  for i in range(15):
    r = np.random.randint(0, high=len(real), size=1)
    set1.append(real[r[0]])
    r = np.random.randint(0, high=len(fake), size=1)
    set2.append(fake[r[0]])
    r = np.random.randint(0, high=len(real), size=1)
    set1.append(real[r[0]])
    r = np.random.randint(0, high=len(fake), size=1)
    set2.append(real[r[0]])
    Y.append(-1)
    Y.append(1)
set1 = np.asarray(set1)
set2 = np.asarray(set2)
Y = np.asarray(Y)
Y_fake = np.zeros(()) 
print(set1.shape, set2.shape, Y.shape)

(150, 50, 125, 3) (150, 50, 125, 3) (150,)


In [ ]:
##2
def loss_fnc(y_true, y_pred):
  return tf.reduce_sum(tf.multiply(y_pred,y_true))

In [ ]:
premodel.trainable = False
added_model.trainable = False

In [ ]:
def comparor():
  inp1 = keras.layers.Input(shape = (50,125, 3))
  inp2 = keras.layers.Input(shape = (50,125, 3))
  feature1 = premodel(inp1)
  feature1 = keras.layers.Flatten()(feature1)
  feature1 = added_model(feature1)
  feature2 = premodel(inp2)
  feature2 = keras.layers.Flatten()(feature2)
  feature2 = added_model(feature2)

  concat = keras.layers.Lambda(lambda x : tf.concat(x, axis = -1))([feature1, feature2])
  out = keras.layers.Dense(100, activation = 'elu')(concat)
  out = keras.layers.Dense(50, activation = 'elu')(out)
  out = keras.layers.Dense(10, activation = 'elu')(out)
  closeness = keras.layers.Dense(1, activation = 'sigmoid')(out)
  #diff = keras.layers.Lambda(lambda x : tf.math.square(tf.math.subtract(x[0] , x[1])))([feature1, feature2])
  model = keras.Model([inp1, inp2], closeness)
  return model

In [ ]:
comparor = comparor()

In [ ]:
comparor.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_6 (InputLayer)            [(None, 50, 125, 3)] 0                                            
__________________________________________________________________________________________________
input_7 (InputLayer)            [(None, 50, 125, 3)] 0                                            
__________________________________________________________________________________________________
vgg16 (Model)                   (None, 1, 3, 512)    14714688    input_6[0][0]                    
                                                                 input_7[0][0]                    
__________________________________________________________________________________________________
flatten_3 (Flatten)             (None, 1536)         0           vgg16[4][0]                

In [ ]:
##4
comparor.compile(optimizer = 'Adam', loss = loss_fnc)

In [ ]:
##3
comparor = keras.models.load_model('/content/drive/My Drive/SOAI/Ass 2 /comparor.hdf5', custom_objects={'loss_fnc':loss_fnc})

In [ ]:
comparor.fit([set1, set2], Y, batch_size = set1.shape[0], epochs = 500)
comparor.save('/content/drive/My Drive/SOAI/Ass 2 /comparor.hdf5')

Epoch 1/500


KeyboardInterrupt: ignored

In [ ]:
correct = 0
incorrect = 0
for i in range(Y.shape[0]):
  pred = comparor.predict([set1[i:i+1], set2[i:i+1]])
  if pred[0,0] <=  0.5 :
    if Y[i] == 1:
      correct +=1
    else:
      incorrect +=1
  else :
    if Y[i] == -1:
      correct +=1
    else:
      incorrect +=1
print('Accuracy : ' , correct/(correct+incorrect))  

###Test Set Preparation

In [ ]:
set1 = []
set2 = []

Y = []
# real vs real
for real, fake in zip([real001, real002, real003,real004, real005], [fake001, fake002, fake003, fake004, fake005]):
  for i in range(5):
    r = np.random.randint(0, high=len(real), size=1)
    set1.append(real[r[0]])
    r = np.random.randint(0, high=len(fake), size=1)
    set2.append(fake[r[0]])
    r = np.random.randint(0, high=len(real), size=1)
    set1.append(real[r[0]])
    r = np.random.randint(0, high=len(fake), size=1)
    set2.append(real[r[0]])
    Y.append(-1)
    Y.append(1)
set1 = np.asarray(set1)
set2 = np.asarray(set2)
Y = np.asarray(Y)
Y_fake = np.zeros(()) 
print(set1.shape, set2.shape, Y.shape)

(50, 50, 125, 3) (50, 50, 125, 3) (50,)


In [ ]:
correct = 0
incorrect = 0
for i in range(Y.shape[0]):
  pred = comparor.predict([set1[i:i+1], set2[i:i+1]])
  if pred[0,0] <=  0.5 :
    print('PREDICTED : real')
    if Y[i] == 1:
      correct +=1
    else:
      incorrect +=1
  else :
    print('PREDICTED : fake')
    if Y[i] == -1:
      correct +=1
    else:
      incorrect +=1
  if Y[i] == 1:
    print('ACTUAl : real')
  else:
    print('ACTUAL : fake')
print('Accuracy : ' , correct/(correct+incorrect))  

PREDICTED : fake
ACTUAL : fake
PREDICTED : real
ACTUAl : real
PREDICTED : fake
ACTUAL : fake
PREDICTED : real
ACTUAl : real
PREDICTED : fake
ACTUAL : fake
PREDICTED : real
ACTUAl : real
PREDICTED : fake
ACTUAL : fake
PREDICTED : real
ACTUAl : real
PREDICTED : fake
ACTUAL : fake
PREDICTED : real
ACTUAl : real
PREDICTED : fake
ACTUAL : fake
PREDICTED : real
ACTUAl : real
PREDICTED : fake
ACTUAL : fake
PREDICTED : real
ACTUAl : real
PREDICTED : fake
ACTUAL : fake
PREDICTED : real
ACTUAl : real
PREDICTED : fake
ACTUAL : fake
PREDICTED : real
ACTUAl : real
PREDICTED : fake
ACTUAL : fake
PREDICTED : real
ACTUAl : real
PREDICTED : fake
ACTUAL : fake
PREDICTED : real
ACTUAl : real
PREDICTED : fake
ACTUAL : fake
PREDICTED : real
ACTUAl : real
PREDICTED : fake
ACTUAL : fake
PREDICTED : real
ACTUAl : real
PREDICTED : fake
ACTUAL : fake
PREDICTED : real
ACTUAl : real
PREDICTED : fake
ACTUAL : fake
PREDICTED : real
ACTUAl : real
PREDICTED : fake
ACTUAL : fake
PREDICTED : real
ACTUAl : real
PREDICTE

In [ ]:
##5
set1 = [ cv2.resize(cv2.imread('path to the real signature image of the person'), shape, interpolation = cv2.INTER_AREA)]
set1 = np.asarray(set1)
set2 = [cv2.resize(cv2.imread('path to the image to be tested'), shape, interpolation = cv2.INTER_AREA)]
set2 = np.asarray(set2)


pred = comparor.predict([set1, set2])
if pred[0,0] <=  0.5 :
  print('PREDICTED : real')
else:    
  print('PREDICTED : fake')


In [ ]:
a = np.zeros((10))
print(a.shape)
a = [a]
a = np.asarray(a)
print(a.shape)

(10,)
(1, 10)


###NOTE : The model is showing 100 percent accuracy as it is trained on a very limited dataset(I was not able to find a bigger publicly available dataset). But the way in which it is trained ensures that it works fine for any other dataset probably with condition of black ink on white background(as all images in training data were in sam format).


 # *FAILED ATTEMPTS*

#CNN LSTM BASED APPROACH
was not able to differentiate between the real and fake signatures. The reason is not clear, Maybe the LSTM design does not capture the differentiating aspect of a signature.

In [ ]:
set1 = []
set2 = []
Y = []
# real vs real
for real, fake in zip([real001, real002, real003,real004, real005], [fake001, fake002, fake003, fake004, fake005]):
  for i in range(20):
    r = np.random.randint(0, high=len(real), size=2)
    set1.append(real[r[0]])
    set2.append(real[r[1]])
    Y.append([1,0])
    r1 = np.random.randint(0, high=len(real), size=1)
    r2 = np.random.randint(0, high=len(fake), size=1)
    if (i < 10):
      set1.append(real[r1[0]])
      set2.append(fake[r2[0]])
      Y.append([0,1])
    else:
      set1.append(fake[r2[0]])
      set2.append(real[r1[0]])
      Y.append([0,1])

set1 = np.asarray(set1)
set2 = np.asarray(set2)
Y = np.asarray(Y)
#set1 = set1/255 - 0.5
#set2 = set2/255 - 0.5
print(set1.shape, set2.shape, Y.shape, Y)


In [ ]:
def encoder(height, width):
  inp1 = keras.Input(shape = (height, width,3))
  inp2 = keras.Input(shape = (height, width,3))
  l1 = keras.layers.Conv2D(32, (height, 5), activation = 'elu')
  l2 = keras.layers.Conv2D(64, (32, 5), activation = 'elu')
  l3 = keras.layers.Conv2D(128, (64, 5), activation = 'elu')
  l4 = keras.layers.Conv2D(256, (128, 5), activation = 'elu')
  l5 = keras.layers.Conv2D(512, (256, 5), activation = 'elu')
  bi1 = keras.layers.Bidirectional(keras.layers.LSTM(100, return_sequences=True), merge_mode="concat")

  en1 = l1(inp1)
  en1 = keras.layers.Lambda(lambda x:tf.squeeze(x, [-3]))(en1)
  en1 = keras.layers.AveragePooling1D(pool_size=2, strides=2, padding="valid")(en1)
  # (none, steps/2, 32)
  en1 = keras.layers.Lambda(lambda x: tf.expand_dims(tf.transpose(x, [0,2,1]), -1))(en1)
  # (none, 32, steps/2,1)
  
  en1 = l2(en1)
  en1 = keras.layers.Lambda(lambda x:tf.squeeze(x, [-3]))(en1)
  #(none, steps/2, 64)
  #en1 = keras.layers.AveragePooling1D(pool_size=2, strides=2, padding="valid")(en1)

  en1 = keras.layers.Lambda(lambda x: tf.expand_dims(tf.transpose(x, [0,2,1]), -1))(en1)
  en1 = l3(en1)
  en1 = keras.layers.Lambda(lambda x:tf.squeeze(x, [-3]))(en1)

  en1 = keras.layers.Lambda(lambda x: tf.expand_dims(tf.transpose(x, [0,2,1]), -1))(en1)
  en1 = l4(en1)
  en1 = keras.layers.Lambda(lambda x:tf.squeeze(x, [-3]))(en1)

  en1 = keras.layers.Lambda(lambda x: tf.expand_dims(tf.transpose(x, [0,2,1]), -1))(en1)
  en1 = l5(en1)
  en1 = keras.layers.Lambda(lambda x:tf.squeeze(x, [-3]))(en1)
  
  en1 = bi1(en1)

  en2 = l1(inp2)
  en2 = keras.layers.Lambda(lambda x:tf.squeeze(x, [-3]))(en2)
  en2 = keras.layers.AveragePooling1D(pool_size=2, strides=2, padding="valid")(en2)
  # (none, steps/2, 32)
  en2 = keras.layers.Lambda(lambda x: tf.expand_dims(tf.transpose(x, [0,2,1]), -1))(en2)
  # (none, 32, steps/2,1)
  
  en2 = l2(en2)
  en2 = keras.layers.Lambda(lambda x:tf.squeeze(x, [-3]))(en2)
  #(none, steps/2, 64)
  #en1 = keras.layers.AveragePooling1D(pool_size=2, strides=2, padding="valid")(en1)

  en2 = keras.layers.Lambda(lambda x: tf.expand_dims(tf.transpose(x, [0,2,1]), -1))(en2)
  en2 = l3(en2)
  en2 = keras.layers.Lambda(lambda x:tf.squeeze(x, [-3]))(en2)

  en2 = keras.layers.Lambda(lambda x: tf.expand_dims(tf.transpose(x, [0,2,1]), -1))(en2)
  en2 = l4(en2)
  en2 = keras.layers.Lambda(lambda x:tf.squeeze(x, [-3]))(en2)

  en2 = keras.layers.Lambda(lambda x: tf.expand_dims(tf.transpose(x, [0,2,1]), -1))(en2)
  en2 = l5(en2)
  en2 = keras.layers.Lambda(lambda x:tf.squeeze(x, [-3]))(en2)
  
  en2 = bi1(en2)

  model = keras.Model([inp1, inp2], [en1, en2])
  return model


In [ ]:
enc = encoder(shape[1], shape[0])
enc.summary()

In [ ]:
def discriminator():
  inp1 = keras.Input(shape = enc.layers[-1].output_shape[1:])
  inp2 = keras.Input(shape = enc.layers[-1].output_shape[1:])
  con = keras.layers.Lambda(lambda x: tf.concat(x, axis = 2))([inp1, inp2])
  lstm = keras.layers.Bidirectional(keras.layers.LSTM(50, return_sequences=True), merge_mode = 'concat')(con)
  lstm = keras.layers.Bidirectional(keras.layers.LSTM(25, return_sequences=True), merge_mode = 'concat')(lstm)
  lstm = keras.layers.Bidirectional(keras.layers.LSTM(10, return_sequences=True), merge_mode = 'concat')(lstm)
  fl = keras.layers.Flatten()(lstm)
  out = keras.layers.Dense(1000, activation = 'elu')(fl)
  out = keras.layers.Dense(100,activation = 'elu')(out)
  out = keras.layers.Dense(10, activation = 'elu')(out)
  out = keras.layers.Dense(2, activation = 'softmax')(out)
  return keras.Model([inp1, inp2], out)


In [ ]:
dis = discriminator()
dis.summary()

In [ ]:
M = keras.Model(enc.input, dis(enc.output))
M.summary()

In [ ]:
optimizer = keras.optimizers.Adam(learning_rate = 0.001)
M.compile(loss = 'categorical_crossentropy', optimizer = optimizer, metrics = ['accuracy'])

In [ ]:
for i in range(20000):
  print(i)
  M.fit([set1, set2], Y, batch_size = set1.shape[0])
  print(M.predict([set1[0:2], set2[0:2]]))
  print(Y[0:2])
  M.save('/content/drive/My Drive/SOAI/Ass 2 /M.hdf5')

#BUILDING THROUGH CORE TENSORFLOW
Did not work out as the calculation of derivative with respect to loss came out to be nan. might be due to huge loss metric which is mean squared error of 500 dimensional vector.

In [ ]:
resnet = keras.applications.VGG16(include_top=False,weights="imagenet", input_shape=(50,125,3))
D1 = keras.layers.Dense(500, activation = 'sigmoid')

In [ ]:
def model():
  inp1 = keras.layers.Input(shape=(50,125,3))
  inp2 = keras.layers.Input(shape=(50,125,3))

  out1 = resnet(inp1)
  out1 = keras.layers.Flatten()(out1)
  out1 = D1(out1)

  out2 = resnet(inp2)
  out2 = keras.layers.Flatten()(out2)
  out2 = D1(out2)
  
  model = keras.Model([inp1, inp2], [out1, out2])
  return model

In [ ]:
def loss_fn(out_same, out_diff):
  loss_same = tf.reduce_mean(tf.math.square(tf.math.subtract(out_same[0] , out_same[1])))
  loss_diff = -tf.reduce_mean(tf.math.square(tf.math.subtract(out_diff[0] , out_diff[1])))
  loss = loss_diff + loss_same
  loss = 1/(1+np.exp(loss))
  return loss

In [ ]:
optimizer = keras.optimizers.Adam(learning_rate = 0.001)
set1 = tf.convert_to_tensor(set1, dtype=tf.float32)
set2 = tf.convert_to_tensor(set2, dtype=tf.float32)
set3 = tf.convert_to_tensor(set3, dtype=tf.float32)
for i in range(100):
  with tf.GradientTape() as Tape:
    out_same = m([set1[0:1], set2[0:1]])
    out_diff = m([set1, set3])
    loss = loss_fn(out_same,  out_diff)
  grad = Tape.gradient(loss, m.trainable_variables)
  #grad=tf.clip_by_value(grad, -20, 20)
  print(grad)
  optimizer.apply_gradients(zip(grad, m.trainable_variables))


[<tf.Tensor: shape=(3, 3, 3, 64), dtype=float32, numpy=
array([[[[nan, nan, nan, ..., nan, nan, nan],
         [nan, nan, nan, ..., nan, nan, nan],
         [nan, nan, nan, ..., nan, nan, nan]],

        [[nan, nan, nan, ..., nan, nan, nan],
         [nan, nan, nan, ..., nan, nan, nan],
         [nan, nan, nan, ..., nan, nan, nan]],

        [[nan, nan, nan, ..., nan, nan, nan],
         [nan, nan, nan, ..., nan, nan, nan],
         [nan, nan, nan, ..., nan, nan, nan]]],


       [[[nan, nan, nan, ..., nan, nan, nan],
         [nan, nan, nan, ..., nan, nan, nan],
         [nan, nan, nan, ..., nan, nan, nan]],

        [[nan, nan, nan, ..., nan, nan, nan],
         [nan, nan, nan, ..., nan, nan, nan],
         [nan, nan, nan, ..., nan, nan, nan]],

        [[nan, nan, nan, ..., nan, nan, nan],
         [nan, nan, nan, ..., nan, nan, nan],
         [nan, nan, nan, ..., nan, nan, nan]]],


       [[[nan, nan, nan, ..., nan, nan, nan],
         [nan, nan, nan, ..., nan, nan, nan],
        

KeyboardInterrupt: ignored